In [1]:
import csv
import requests
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse, parseString


In [2]:
cc_xml = ''
with open('./Conversation_SSML.xml', 'r') as cc_file:
    cc_xml = cc_file.read()

In [3]:
conversation = []
i=1

tree = parseString(cc_xml)
for el in tree.getElementsByTagName('voice'):
    conv_part = ''
    speaker = (dict(el.attributes.items())['name'])
    for node in el.childNodes:
        if node.nodeType == node.TEXT_NODE:
            conv_part = node.data
        elif node.tagName == 'prosody':
            conv_part = conv_part + node.firstChild.data
        else: 
            for pro in node.getElementsByTagName('prosody'):
                conv_part = conv_part + pro.firstChild.data
    part = 'Agent'
    if speaker == 'en-US-JennyNeural' : 
        part = 'Agent'
    else:
        part = 'Customer'
    conversation.append({"text":conv_part, 'id':i, 'role':part, 'participantId':'{}'.format(part)})
    i = i+1
    
#print(conversation)


In [4]:
# Format Conversation for Language Service Analysis

json_d = { "displayName": "Conversation Task Example",
  "analysisInput": {
    "conversations": [
      {
        "conversationItems": conversation ,
        "modality": "text",
        "id": "conversation1",
        "language": "en"
      }
    ]
  },
  "tasks": [             
       {
        "taskName": "analyze 1",
        "kind": "ConversationalSummarizationTask",
        "parameters": {
           "summaryAspects": ["chaptertitle","issue", "resolution", "narrative"]

        }
    }
  ]
}

In [5]:
json_d

{'displayName': 'Conversation Task Example',
 'analysisInput': {'conversations': [{'conversationItems': [{'text': 'Hello, how may I help you today?',
      'id': 1,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "I really need help with my credit card, it's not working at all",
      'id': 2,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'May I please have your first and last name?',
      'id': 3,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "sure it's John, J O H N, Doh, D O E",
      'id': 4,
      'role': 'Customer',
      'participantId': 'Customer'},
     {'text': 'Thank you Mr Doh, can you confirm the last four digits of your account number?',
      'id': 5,
      'role': 'Agent',
      'participantId': 'Agent'},
     {'text': "Which number? Is that the card number or the number on my statement, I don't have a statement in front of me.",
      'id': 6,
      'role': 'Customer',
      'participantId': '

In [6]:
import requests
from requests.structures import CaseInsensitiveDict
import json
import io
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
cc_json = ''
cog_svc_ep = parser.get('openai_api','cog_svc_ep')

url = "{}/language/analyze-conversations/jobs?api-version=2022-10-01-preview".format(cog_svc_ep)

headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.post(url, headers=headers, json=data)

print(resp)


<Response [202]>


In [7]:
url = resp.headers['operation-location']

In [8]:
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.get(url, headers=headers)

out = resp.json()

In [14]:
# Display outputs from Language Cognitive service analysis

for task in out['tasks']['items'][0]['results']['conversations'][0]['summaries']:
    print("{} = {}".format(task['aspect'],task['text']))



chapterTitle = Help with Credit Card
issue = Customer wants to purchase a flight. | Customer's credit card is not working.
resolution = Check the account. | Check the order --> fraud. | Check the payment --> fraud.
narrative = The customer has a problem with his credit card. The agent helps him to confirm the last four digits of his account number. The customer confirms the last purchase and the amount. The agent approves the booking.


In [19]:
narr = out['tasks']['items'][0]['results']['conversations'][0]['summaries'][3]['text'].split('.')

## Open AI Summarization

In [14]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model = parser.get('openai_api','api_model')

In [16]:
prompt

'Generate detailed call summary notes from the following text\nText:\n###\n[{\'text\': \'Hello, how may I help you today?\', \'id\': 1, \'role\': \'Agent\', \'participantId\': \'Agent\'}, {\'text\': "I really need help with my credit card, it\'s not working at all", \'id\': 2, \'role\': \'Customer\', \'participantId\': \'Customer\'}, {\'text\': \'May I please have your first and last name?\', \'id\': 3, \'role\': \'Agent\', \'participantId\': \'Agent\'}, {\'text\': "sure it\'s John, J O H N, Doh, D O E", \'id\': 4, \'role\': \'Customer\', \'participantId\': \'Customer\'}, {\'text\': \'Thank you Mr Doh, can you confirm the last four digits of your account number?\', \'id\': 5, \'role\': \'Agent\', \'participantId\': \'Agent\'}, {\'text\': "Which number? Is that the card number or the number on my statement, I don\'t have a statement in front of me.", \'id\': 6, \'role\': \'Customer\', \'participantId\': \'Customer\'}, {\'text\': \'It should be the last four digits printed on your credit

In [15]:
prompt = "Generate detailed call summary notes from the following text\nText:\n###\n{}\n###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

- Customer is having trouble with their credit card
- Customer confirms their name and account number
- Agent confirms a suspected fraudulent charge with the customer
- Customer books a stay with their credit card
- Agent helps customer order a credit card for their daughter
- Customer provides daughter's name and mailing address


In [11]:
prompt = "Extract list of call participants from the following text\nText:\n###\n{}\n###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

Agent, Customer


In [12]:
prompt = "Generate list of call follow up tasks from following text\nText:\n###\n{}\n###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.7,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

-Confirm last four digits of credit card number with customer
-Confirm last purchase made with customer
-Confirm charge amount with customer
-Order new credit card for customer


In [13]:
print(prompt)

Generate list of call follow up tasks from following text
Text:
###
[{'text': 'Hello, how may I help you today?', 'id': 1, 'role': 'Agent', 'participantId': 'Agent'}, {'text': "I really need help with my credit card, it's not working at all", 'id': 2, 'role': 'Customer', 'participantId': 'Customer'}, {'text': 'May I please have your first and last name?', 'id': 3, 'role': 'Agent', 'participantId': 'Agent'}, {'text': "sure it's John, J O H N, Doh, D O E", 'id': 4, 'role': 'Customer', 'participantId': 'Customer'}, {'text': 'Thank you Mr Doh, can you confirm the last four digits of your account number?', 'id': 5, 'role': 'Agent', 'participantId': 'Agent'}, {'text': "Which number? Is that the card number or the number on my statement, I don't have a statement in front of me.", 'id': 6, 'role': 'Customer', 'participantId': 'Customer'}, {'text': 'It should be the last four digits printed on your credit card.', 'id': 7, 'role': 'Agent', 'participantId': 'Agent'}, {'text': 'Ok, let me get it, 